In [1]:
import numpy as np
import numpy.matlib
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from mpl_toolkits import mplot3d
import matplotlib.animation as animation

## Steady State

In [ ]:
# z = [u, v, w, y]


# Rate Constants
k1aaCT  = 0.015
k2      = 0
k3CT    = 200
k4prime = 0.018
k5tilP  = 0
k7      = 0.6
k8tilP  = 50 # >> k9
k9      = 10 # >> k6


# Steady State Initial Conditions
u0 = 0.001
v0 = 0.24
w0 = 0.001
y0 = 0.26
k4 = 180
k6 = 1

t0 = 0
tf = 100


def steadystate(t, z):
    dudt = k4*(z[2] - z[0])*(k4prime/k4 + (z[0])**2) - k6*z[0]
    dvdt = k1aaCT - k2*(z[1] - z[2]) - k6*z[0]
    dwdt = k3CT*(1 - z[2])*(z[1] - z[2]) - k6*z[0]
    dydt = k1aaCT - k2*(z[1] - z[2]) - k7*(z[3] - z[1])
    
    
    return [dudt, dvdt, dwdt, dydt]

ss = solve_ivp(steadystate, (t0, tf), [u0, v0, w0, y0])


u_1 = ss.y[0]
v_1 = ss.y[1]
w_1 = ss.y[2]
y_1 = ss.y[3]


fig = plt.figure(figsize=(16,10), dpi=80, facecolor='w', edgecolor='k')
fig.suptitle('Steady State Phase Planes', fontsize=16)
ax = fig.add_subplot(2, 2, 1, projection='3d')
ax.plot3D(u_1, v_1, w_1, 'black')
ax.set_xlabel('u')
ax.set_ylabel('v')
ax.set_zlabel('w')
ax.set_title('U-V-W')

ax = fig.add_subplot(2, 2, 2, projection='3d')
ax.plot3D(u_1, v_1, y_1, 'black')
ax.set_xlabel('u')
ax.set_ylabel('v')
ax.set_zlabel('y')
ax.set_title('U-V-Y')

ax = fig.add_subplot(2, 2, 3, projection='3d')
ax.plot3D(v_1, w_1, y_1, 'black')
ax.set_xlabel('v')
ax.set_ylabel('w')
ax.set_zlabel('y')
ax.set_title('V-W-Y')

ax = fig.add_subplot(2, 2, 4, projection='3d')
ax.plot3D(u_1, w_1, y_1, 'black')
ax.set_xlabel('u')
ax.set_ylabel('w')
ax.set_zlabel('y')
ax.set_title('U-W-Y')
plt.show()


## Spontaneous Oscillations

### No Pertubation

In [ ]:
# Spontaneous Oscillations Initial Conditions
u0 = 0.0074
v0 = 0.566
w0 = 0.566
y0 = 0.566
k4 = 180
k6 = 2

t0 = 0
tf = 100


def spontosc(t, z):
    dudt = k4*(z[2] - z[0])*(k4prime/k4 + (z[0])**2) - k6*z[0]
    dvdt = k1aaCT - k2*(z[1] - z[2]) - k6*z[0]
    dwdt = k3CT*(1 - z[2])*(z[1] - z[2]) - k6*z[0]
    dydt = k1aaCT - k2*(z[1] - z[2]) - k7*(z[3] - z[1])
    
    
    return [dudt, dvdt, dwdt, dydt]

osc_a = solve_ivp(spontosc, (t0, tf), [u0, v0, w0, y0])


u_2a = osc_a.y[0]
v_2a = osc_a.y[1]
w_2a = osc_a.y[2]
y_2a = osc_a.y[3]



fig = plt.figure(figsize=(16,10), dpi=80, facecolor='w', edgecolor='k')
fig.suptitle('Spontaneous Oscillations Phase Planes', fontsize=16)
ax = fig.add_subplot(2, 2, 1, projection='3d')
ax.plot3D(u_2a, v_2a, w_2a, 'black')
ax.set_xlabel('u')
ax.set_ylabel('v')
ax.set_zlabel('w')
ax.set_title('U-V-W')

ax = fig.add_subplot(2, 2, 2, projection='3d')
ax.plot3D(u_2a, v_2a, y_2a, 'black')
ax.set_xlabel('u')
ax.set_ylabel('v')
ax.set_zlabel('y')
ax.set_title('U-V-Y')

ax = fig.add_subplot(2, 2, 3, projection='3d')
ax.plot3D(v_2a, w_2a, y_2a, 'black')
ax.set_xlabel('v')
ax.set_ylabel('w')
ax.set_zlabel('y')
ax.set_title('V-W-Y')

ax = fig.add_subplot(2, 2, 4, projection='3d')
ax.plot3D(u_2a, w_2a, y_2a, 'black')
ax.set_xlabel('u')
ax.set_ylabel('w')
ax.set_zlabel('y')
ax.set_title('U-W-Y')
plt.show()

### Small Pertubation

In [ ]:
# Spontaneous Oscillations Initial Conditions
u0 = 0.0074
v0 = 0.6
w0 = 0.6
y0 = 0.566
k4 = 180
k6 = 2

t0 = 0
tf = 100


def spontosc(t, z):
    dudt = k4*(z[2] - z[0])*(k4prime/k4 + (z[0])**2) - k6*z[0]
    dvdt = k1aaCT - k2*(z[1] - z[2]) - k6*z[0]
    dwdt = k3CT*(1 - z[2])*(z[1] - z[2]) - k6*z[0]
    dydt = k1aaCT - k2*(z[1] - z[2]) - k7*(z[3] - z[1])
    
    
    return [dudt, dvdt, dwdt, dydt]

osc_b = solve_ivp(spontosc, (t0, tf), [u0, v0, w0, y0])


u_2b = osc_b.y[0]
v_2b = osc_b.y[1]
w_2b = osc_b.y[2]
y_2b = osc_b.y[3]



fig = plt.figure(figsize=(16,10), dpi=80, facecolor='w', edgecolor='k')
fig.suptitle('Spontaneous Oscillations Phase Planes', fontsize=16)
ax = fig.add_subplot(2, 2, 1, projection='3d')
ax.plot3D(u_2b, v_2b, w_2b, 'black')
ax.set_xlabel('u')
ax.set_ylabel('v')
ax.set_zlabel('w')
ax.set_title('U-V-W')

ax = fig.add_subplot(2, 2, 2, projection='3d')
ax.plot3D(u_2b, v_2b, y_2b, 'black')
ax.set_xlabel('u')
ax.set_ylabel('v')
ax.set_zlabel('y')
ax.set_title('U-V-Y')

ax = fig.add_subplot(2, 2, 3, projection='3d')
ax.plot3D(v_2b, w_2b, y_2b, 'black')
ax.set_xlabel('v')
ax.set_ylabel('w')
ax.set_zlabel('y')
ax.set_title('V-W-Y')

ax = fig.add_subplot(2, 2, 4, projection='3d')
ax.plot3D(u_2b, w_2b, y_2b, 'black')
ax.set_xlabel('u')
ax.set_ylabel('w')
ax.set_zlabel('y')
ax.set_title('U-W-Y')
plt.show()

### Large Pertubation

In [ ]:
# Spontaneous Oscillations Initial Conditions
u0 = 0.0074
v0 = 0.65
w0 = 0.566
y0 = 0.566
k4 = 180
k6 = 2

t0 = 0
tf = 100


def spontosc(t, z):
    dudt = k4*(z[2] - z[0])*(k4prime/k4 + (z[0])**2) - k6*z[0]
    dvdt = k1aaCT - k2*(z[1] - z[2]) - k6*z[0]
    dwdt = k3CT*(1 - z[2])*(z[1] - z[2]) - k6*z[0]
    dydt = k1aaCT - k2*(z[1] - z[2]) - k7*(z[3] - z[1])
    
    
    return [dudt, dvdt, dwdt, dydt]

osc_c = solve_ivp(spontosc, (t0, tf), [u0, v0, w0, y0])


u_2c = osc_c.y[0]
v_2c = osc_c.y[1]
w_2c = osc_c.y[2]
y_2c = osc_c.y[3]



fig = plt.figure(figsize=(16,10), dpi=80, facecolor='w', edgecolor='k')
fig.suptitle('Spontaneous Oscillations Phase Planes with Large Pertubation', fontsize=16)
ax = fig.add_subplot(2, 2, 1, projection='3d')
ax.plot3D(u_2c, v_2c, w_2c, 'black')
ax.set_xlabel('u')
ax.set_ylabel('v')
ax.set_zlabel('w')
ax.set_title('U-V-W')

ax = fig.add_subplot(2, 2, 2, projection='3d')
ax.plot3D(u_2c, v_2c, y_2c, 'black')
ax.set_xlabel('u')
ax.set_ylabel('v')
ax.set_zlabel('y')
ax.set_title('U-V-Y')

ax = fig.add_subplot(2, 2, 3, projection='3d')
ax.plot3D(v_2c, w_2c, y_2c, 'black')
ax.set_xlabel('v')
ax.set_ylabel('w')
ax.set_zlabel('y')
ax.set_title('V-W-Y')

ax = fig.add_subplot(2, 2, 4, projection='3d')
ax.plot3D(u_2c, w_2c, y_2c, 'black')
ax.set_xlabel('u')
ax.set_ylabel('w')
ax.set_zlabel('y')
ax.set_title('U-W-Y')
plt.show()

## Excitable Switch

In [ ]:
# z = [u, v, w, y, k6]

# Excitable Switch Initial Conditions
u0 = 0.0074
v0 = 0.566
w0 = 0.566
y0 = 0.566
k4 = 180


k60 = 2.5

interval = 100
no_repeats = 5

t_a = np.linspace(0, interval, interval*10)
t = np.linspace(0, interval*no_repeats, interval*no_repeats*10)
teval = list(t)

k6 = []
b = 0.03
a = 1.5 + np.exp(-b*t_a)
a = list(a)

for i in range(no_repeats):
    k6.append(a)

k6 = [item for sublist in k6 for item in sublist]


t0 = 0
tf = 500


def switch(t, z):
    dudt = k4*(z[2] - z[0])*(k4prime/k4 + (z[0])**2) - z[4]*z[0]
    dvdt = k1aaCT - k2*(z[1] - z[2]) - z[4]*z[0]
    dwdt = k3CT*(1 - z[2])*(z[1] - z[2]) - z[4]*z[0]
    dydt = k1aaCT - k2*(z[1] - z[2]) - k7*(z[3] - z[1])
    dk6  = 0.075*np.exp(-b*t)
        
    
    
    return [dudt, dvdt, dwdt, dydt, dk6]

swi = solve_ivp(switch, (t0, tf), [u0, v0, w0, y0, k60])


u_3 = swi.y[0]
v_3 = swi.y[1]
w_3 = swi.y[2]
y_3 = swi.y[3]

fig = plt.figure(figsize=(16,10), dpi=80, facecolor='w', edgecolor='k')
fig.suptitle('Excitable Switch Phase Planes', fontsize=16)
ax = fig.add_subplot(2, 2, 1, projection='3d')
ax.plot3D(u_3, v_3, w_3, 'black')
ax.set_xlabel('u')
ax.set_ylabel('v')
ax.set_zlabel('w')
ax.set_title('U-V-W')

ax = fig.add_subplot(2, 2, 2, projection='3d')
ax.plot3D(u_3, v_3, y_3, 'black')
ax.set_xlabel('u')
ax.set_ylabel('v')
ax.set_zlabel('y')
ax.set_title('U-V-Y')

ax = fig.add_subplot(2, 2, 3, projection='3d')
ax.plot3D(v_3, w_3, y_3, 'black')
ax.set_xlabel('v')
ax.set_ylabel('w')
ax.set_zlabel('y')
ax.set_title('V-W-Y')

ax = fig.add_subplot(2, 2, 4, projection='3d')
ax.plot3D(u_3, w_3, y_3, 'black')
ax.set_xlabel('u')
ax.set_ylabel('w')
ax.set_zlabel('y')
ax.set_title('U-W-Y')
plt.show()

In [ ]:
fig = plt.figure(figsize=(15.5,1), dpi=80, facecolor='w', edgecolor='k')
ax = fig.add_subplot(1, 3, 1)
ax = fig.add_subplot(1, 3, 2)
ax = fig.add_subplot(1, 3, 3)
ax.plot(t, k6)
ax.set_xlabel('Time (mins)')
ax.set_ylabel('k6 ($min^{-1}$)')

plt.show()

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18,5), dpi=80, facecolor='w', edgecolor='k')
ax1.plot(ss.t, u_1, label='[M]/[CT]')
ax1.plot(ss.t, y_1, label='[YT]/[CT]')
ax1.set_xlabel('Time (mins)')
ax1.legend(loc='upper right')
ax1.set_ylabel('Concentration Ratio')
ax1.set_title('Steady State')

ax2.plot(osc_c.t, u_2c, label='[M]/[CT]')
ax2.plot(osc_c.t, y_2c, label='[YT]/[CT]')
ax2.set_xlabel('Time (mins)')
ax2.legend(loc='upper right')
ax2.set_ylabel('Concentration Ratio')
ax2.set_yscale('log')
ax2.set_title('Spontaneous Oscillations')

ax3.plot(swi.t, u_3, label='[M]/[CT]')
ax3.plot(swi.t, y_3, label='[YT]/[CT]')
ax3.set_xlabel('Time (mins)')
ax3.legend(loc='upper right')
ax3.set_ylabel('Concentration Ratio')
ax3.set_title('Excitable Switch')

plt.show()

## Spontaneous Oscillator Comparison

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18,5), dpi=80, facecolor='w', edgecolor='k')
ax1.plot(osc_a.t, u_2a, label='[M]/[CT]')
ax1.plot(osc_a.t, y_2a, label='[YT]/[CT]')
ax1.set_xlabel('Time (mins)')
ax1.legend(loc='upper right')
ax1.set_ylabel('Concentration Ratio')
ax1.set_yscale('log')
ax1.set_title('No Pertubation')

ax2.plot(osc_b.t, u_2b, label='[M]/[CT]')
ax2.plot(osc_b.t, y_2b, label='[YT]/[CT]')
ax2.set_xlabel('Time (mins)')
ax2.legend(loc='upper right')
ax2.set_ylabel('Concentration Ratio')
ax2.set_yscale('log')
ax2.set_title('Small Pertubation')

ax3.plot(osc_c.t, u_2c, label='[M]/[CT]')
ax3.plot(osc_c.t, y_2c, label='[YT]/[CT]')
ax3.set_xlabel('Time (mins)')
ax3.legend(loc='upper right')
ax3.set_ylabel('Concentration Ratio')
ax3.set_yscale('log')
ax3.set_title('Large Pertubation')

plt.show()


fig = plt.figure(figsize=(16,10), dpi=80, facecolor='w', edgecolor='k')
fig.suptitle('Spontaneous Oscillation Pertubation Comparison', fontsize=16)
ax = plt.axes(projection='3d')
ax.plot3D(u_1, v_1, w_1, label='Steady State')
ax.plot3D(u_2a, v_2a, w_2a, label='Spontaneous Oscillator - No Pert')
ax.plot3D(u_2b, v_2b, w_2b, label='Spontaneous Oscillator - Small Pert')
ax.plot3D(u_2c, v_2c, w_2c, label='Spontaneous Oscillator - Large Pert')
ax.set_xlabel('u')
ax.set_ylabel('v')
ax.set_zlabel('w')
ax.legend()

In [ ]:
fig = plt.figure(figsize=(16,10), dpi=80, facecolor='w', edgecolor='k')
fig.suptitle('Steady State - Spontaneous Oscillation with Large Pertubation Comparison', fontsize=16)
ax = fig.add_subplot(2, 2, 1, projection='3d')
ax.plot3D(u_1, v_1, w_1, label='Steady State')
ax.plot3D(u_2c, v_2c, w_2c, label='Spontaneous Oscillation')
ax.set_xlabel('u')
ax.set_ylabel('v')
ax.set_zlabel('w')
ax.legend(loc='upper right')
ax.set_title('U-V-W')

ax = fig.add_subplot(2, 2, 2, projection='3d')
ax.plot3D(u_1, v_1, y_1, label='Steady State')
ax.plot3D(u_2c, v_2c, y_2c, label='Spontaneous Oscillation')
ax.set_xlabel('u')
ax.set_ylabel('v')
ax.set_zlabel('y')
ax.legend(loc='upper right')
ax.set_title('U-V-Y')

ax = fig.add_subplot(2, 2, 3, projection='3d')
ax.plot3D(v_1, w_1, y_1, label='Steady State')
ax.plot3D(v_2c, w_2c, y_2c, label='Spontaneous Oscillation')
ax.set_xlabel('v')
ax.set_ylabel('w')
ax.set_zlabel('y')
ax.legend(loc='upper right')
ax.set_title('V-W-Y')

ax = fig.add_subplot(2, 2, 4, projection='3d')
ax.plot3D(u_1, w_1, y_1, label='Steady State')
ax.plot3D(u_2c, w_2c, y_2c, label='Spontaneous Oscillation')
ax.set_xlabel('u')
ax.set_ylabel('w')
ax.set_zlabel('y')
ax.legend(loc='upper right')
ax.set_title('U-W-Y')
plt.show()